## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
  - оценка 2 и меньше — низкий рейтинг
  - оценка 4 и меньше — средний рейтинг
  - оценка 4.5 и 5 — высокий рейтинг

Результат классификации запишите в столбец class

## Решение:

In [234]:
import pandas as pd

In [235]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [236]:
def define_class(rating):
    if rating <= 2:
        return 'low'
    elif rating <= 4:
        return 'medium'
    else:
        return  'high'

In [237]:
ratings['class'] = ratings['rating'].apply(define_class)

In [238]:
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,medium
1,1,1029,3.0,1260759179,medium
2,1,1061,3.0,1260759182,medium
3,1,1129,2.0,1260759185,low
4,1,1172,4.0,1260759205,medium


## Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Результат классификации запишите в отдельный столбец region.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

## Решение 1:

In [240]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [241]:
keywords = pd.read_csv('keywords.csv')

In [242]:
def define_region(keyword, geo_data):
    for region, cities in geo_data.items():
        for city in cities:
            if keyword.find(city) != -1:
                return region
    return 'undefined'

In [243]:
keywords['region'] = keywords['keyword'].apply(define_region, geo_data = geo_data)

In [244]:
keywords[keywords['region'] == 'Дальний Восток'].head()

,keyword,shows,region
1063,фарпост владивосток,176951,Дальний Восток
2175,дром хабаровск,98361,Дальний Восток
3206,дром владивосток,73084,Дальний Восток
3275,100 сп владивосток,76418,Дальний Восток
3687,авито хабаровск,63736,Дальний Восток


## Задание 3 (бонусное)

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

## Решение:

In [245]:
movies = pd.read_csv('ml-latest-small/movies.csv')

In [246]:
years = list(range(1950,2017))

In [247]:
ratings_extended = ratings.merge(movies, on='movieId', how='left')
ratings_extended.head()

,userId,movieId,rating,timestamp,class,title,genres
0,1,31,2.5,1260759144,medium,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,medium,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,medium,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,low,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,medium,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [248]:
def production_year(title, years):
    for year in years:
        if title.find(str(year)) != -1:
            return year
    return 1900

In [249]:
ratings_extended['year'] = ratings_extended['title'].apply(production_year, years=years)

In [250]:
ratings_extended.head()

,userId,movieId,rating,timestamp,class,title,genres,year
0,1,31,2.5,1260759144,medium,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,medium,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,medium,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,low,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,medium,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [251]:
ratings_extended.groupby('year').mean()['rating'].reset_index().sort_values('rating', ascending=False)

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
...,...,...
62,2011,3.401188
66,2015,3.371028
64,2013,3.357399
51,2000,3.356104
